In [8]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

import json
import os
import tensorflow as tf
import pickle
import numpy as np

In [9]:
import config as c
import coco_helper as ch

In [10]:
import matplotlib.pyplot as plt

----

In [11]:
IMAGE_HEIGHT=300
IMAGE_WIDTH=300
IMAGE_SIZE=(IMAGE_HEIGHT,IMAGE_WIDTH)
GAUSSIAN_SPOT_SIGMA_SQ=0.005
JOINT_WIDTH=0.01

In [12]:
from IPython.display import Image, display
def show_by_id(id):
    display(Image(filename=ch.id_to_filename(id)))
def image_by_id(id,resize=True):
    f=tf.io.read_file(ch.id_to_filename(id))
    img= tf.image.decode_jpeg(f)
    img= tf.image.convert_image_dtype(img,tf.float32)
    if resize:
        img=tf.image.resize(img,IMAGE_SIZE)
    return img

In [13]:
def to_3_channels(one_channel,channel=0):
    def rotate(l, x):
        return l[-x:] + l[:-x]
    zeros=np.zeros_like(one_channel)
    channels=(one_channel,zeros,zeros)
    return np.stack(rotate(channels,channel),axis=-1)

-----

In [14]:
with open(c.TRANSFORMED_TRAIN_ANNOTATIONS_PATH, 'rb') as f:
    combined_dict = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '.\\dataset\\transformed\\person_keypoints_train2017'

In [ ]:
ids=[]
sizes=[]
keypoints=[]
joints=[]
for idd,l in combined_dict.items():
    ids.append(idd)
    sizes.append(l[0])
    keypoints.append(l[1])
    joints.append(l[2])

In [ ]:
rt_keypoints=tf.ragged.constant(keypoints)

In [ ]:
rt_joints=tf.ragged.constant(joints)

In [ ]:
ds=tf.data.Dataset.from_tensor_slices((ids, sizes,rt_keypoints,rt_joints))

---

In [ ]:
it=iter(ds)

In [ ]:
sample_tensor=next(it)
sample_tensor

In [ ]:
#constants
x_grid=tf.linspace(0.0,1.0,IMAGE_WIDTH) 
y_grid=tf.linspace(0.0,1.0,IMAGE_HEIGHT)

xx,yy=tf.meshgrid(x_grid,y_grid)
grid=tf.stack((xx,yy),axis=-1)


@tf.function
def draw_spots_v4(kpts_tensor):
    kpts_tensor=kpts_tensor.to_tensor()
    results = tf.TensorArray(tf.float32, size=17)
    for i in tf.range(17):
        kpts_layer = kpts_tensor[i]
        total_dist=tf.ones(IMAGE_SIZE,dtype=tf.float32)

        for kpt in kpts_layer:
            if kpt[2]==tf.constant(0.0):
                continue
            #must add condition to deal with zeros
            ortho_dist=grid-kpt[0:2] 
            spot_dist=tf.linalg.norm(ortho_dist,axis=-1)         
            total_dist=tf.math.minimum(spot_dist,total_dist)

        results=results.write(i, total_dist)
    raw=tf.exp((-(results.stack()**2)/GAUSSIAN_SPOT_SIGMA_SQ))
    return tf.where(raw < 0.001, 0.0, raw)

In [ ]:
sample_tensor[2][0]

In [ ]:
spots=draw_spots_v4(sample_tensor[2])

In [ ]:
plt.imshow(image_by_id(sample_tensor[0].numpy()))

In [ ]:
plt.imshow(spots.numpy().max(axis=0))

In [ ]:
plt.imshow(image_by_id(sample_tensor[0].numpy())+to_3_channels(spots.numpy().max(axis=0)))

In [ ]:
to_3_channels(spots.numpy().max(axis=0)).shape

# SUCCESS!!!

testing

In [ ]:
test=tf.ragged.constant([[[0.5197505354881287, 0.3812499940395355, 2.0]], [[0.5509355664253235, 0.34843748807907104, 2.0]], [[0.48856547474861145, 0.3671875, 2.0]], [[0.6424116492271423, 0.35468751192092896, 2.0]], [[0.48856547474861145, 0.39531248807907104, 2.0]], [[0.7380457520484924, 0.526562511920929, 2.0]], [[0.4469854533672333, 0.534375011920929, 2.0]], [[0.8461538553237915, 0.7718750238418579, 2.0]], [[0.442827433347702, 0.8125, 2.0]], [[0.9251559376716614, 0.964062511920929, 2.0]], [[0.5072765350341797, 0.698437511920929, 2.0]], [[0.7027027010917664, 0.942187488079071, 2.0]], [[0.55509352684021, 0.949999988079071, 2.0]], [[0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0]], [[0.0, 0.0, 0.0]]])

In [ ]:
r=draw_spots_v4(test)
r[13][150][100]

In [ ]:
plt.imshow(r.numpy()[0])

In [ ]:
plt.imshow(image_by_id(36))

----
Succesful first version, converting to map_fn

In [ ]:
y_grid=tf.linspace(0.0,1.0,IMAGE_HEIGHT)
x_grid=tf.linspace(0.0,1.0,IMAGE_WIDTH) 

xx,yy=tf.meshgrid(x_grid,y_grid)
grid=tf.stack((xx,yy),axis=-1)

@tf.function
def draw_spots_v5(kpts_tensor):
    kpts_tensor=kpts_tensor.to_tensor() #seems to be mandatory for map_fn
    all_dists=tf.map_fn(draw_layer, kpts_tensor) #,parallel_iterations=20) for cpu it has no difference, maybe for gpu it will
    
    raw=tf.exp((-(all_dists**2)/c.GAUSSIAN_SPOT_SIGMA_SQ))
    return tf.where(raw < 0.001, 0.0, raw)

@tf.function
def draw_layer(kpts_layer):    
    layer_dists=tf.map_fn(kpt_dist, kpts_layer)
    return tf.math.reduce_min(layer_dists,axis=0)

@tf.function
def kpt_dist(kpt):
    if kpt[2]==tf.constant(0.0):
        return tf.ones(IMAGE_SIZE,dtype=tf.float32) #maximum distance incase of empty kpt, not ideal but meh
    else:
        ortho_dist=grid-kpt[0:2] 
        return tf.linalg.norm(ortho_dist,axis=-1)
        

In [ ]:
%%timeit -n 10
draw_spots_v4(test)

In [ ]:
%%timeit -n 10
draw_spots_v5(test)

In [ ]:
#checking that the output is simliar
(draw_spots_v4(test)==draw_spots_v5(test)).numpy().all()

Suprisngly both functions are about the same time, with slight advantage to v5, I guess for a cpu this potential parrallism is not significant, maybe it's better on gpu or tpu.
either way great result!

---

# Making PAFs

In [ ]:
y_grid=tf.linspace(0.0,1.0,IMAGE_HEIGHT)
x_grid=tf.linspace(0.0,1.0,IMAGE_WIDTH) 

xx,yy=tf.meshgrid(x_grid,y_grid)
grid=tf.stack((xx,yy),axis=-1)

@tf.function
def draw_PAF_v1(joints_tensor):
    joints_tensor=joints_tensor.to_tensor() #seems to be mandatory for map_fn
    all_pafs=tf.map_fn(PAF_layer, joints_tensor) #,parallel_iterations=20) for cpu it has no difference, maybe for gpu it will
    #this must be executed in the packing order, to produce the layers in the right order
    
    return tf.stack(all_pafs)

@tf.function
def PAF_layer(joints):
    #Makes a combined PAF for all joints of the same type
    layer_PAFS=tf.map_fn(single_joint, joints)
    return tf.math.reduce_mean(layer_PAFS,axis=0) #averages the vectors out to combine the fields in case they intersect

@tf.function
def single_joint(joint):
    #Makes a single vector valued PAF (part affinity field) array
    jpts=tf.reshape(joint[0:4],(2,2))  #reshape to ((x1,y1),(x2,y2))
    if joint[4]==tf.constant(0.0):
        return tf.zeros((IMAGE_HEIGHT,IMAGE_WIDTH,2),dtype=tf.float32) #in case of empty joint
    else:
        #this follows the OpenPose paper ofr generating the PAFs
        vector_full=jpts[1]-jpts[0] #get the joint vector
        vector_length=tf.linalg.norm(vector_full) #get joint length
        vector_hat=vector_full/vector_length  #get joint unit vector 
        
        grid_vectors=grid-jpts[0]  #get grid of vectors from first joint point
        projections=tf.tensordot(grid_vectors,vector_hat,1) #get projection on the joint unit vector
        
        normal_vector=tf.stack((-vector_hat[1],vector_hat[0]))
        n_projections=tf.tensordot(grid_vectors,normal_vector,1) #get projection on the joint normal unit vector
        na_projections=tf.abs(n_projections) #absolute value to get both sides of rhe joint
        
        limit=(0<=projections) & (projections<=vector_length) & (na_projections<=JOINT_WIDTH)
        
        limit_brdcst=tf.stack((limit,limit),axis=-1) #this is for broadcasting to the 2 tuple
        
        return tf.where(limit_brdcst,vector_hat,tf.constant((0.0,0.0)))

Testing a single joint PAF

In [ ]:
joint1=tf.constant((0.1,0.1,0.7,0.7,2.0))
joint2=tf.constant((0.7,0.1,0.2,0.7,2.0))
joint3=tf.constant((0.1,0.3,0.7,0.9,2.0))
joints=tf.stack((joint1,joint2,joint3))
#joints

In [ ]:
JOINT_WIDTH=0.005 #for testing #probably better to leave it bigger

In [ ]:
paf1=single_joint(joint1)
paf2=single_joint(joint2)
paf3=single_joint(joint3)

In [ ]:
def plot_vector_field(paf,downsample=5):
    """if downsample is 1, original size is returned"""
    plt.figure(figsize=(8,8))
    if downsample:
        U=paf[::downsample,::downsample,0]
        V=paf[::downsample,::downsample,1]
    plt.quiver(U,V,scale=5,angles="xy")

In [ ]:
plot_vector_field(paf1,downsample=10)

Sucess for single PAF

In [ ]:
plot_vector_field(PAF_layer(joints))

Success!!

In [ ]:
all_pafs=draw_PAF_v1(sample_tensor[3])
plot_vector_field(all_pafs[0])

In [ ]:
abs(all_pafs.numpy()).max(axis=0).shape

In [ ]:
def plot_vector_field_img(paf,downsample=5):
    """if downsample is 1, original size is returned"""
    plt.figure(figsize=(8,8))
    if downsample:
        U=paf[::downsample,::downsample,0]
        V=paf[::downsample,::downsample,1]
    plt.quiver(U,V,scale=20,angles="xy",minlength=0.1,linewidth=0.1,color='r')
plot_vector_field_img(abs(all_pafs.numpy()).max(axis=0),downsample=1)
plt.imshow(image_by_id(sample_tensor[0]))
plt.show()

In [ ]:
import visualizations as v

In [ ]:
import importlib
importlib.reload(v)

In [ ]:
v.plot_PAFs_on_img(all_pafs.numpy(),image_by_id(sample_tensor[0]),downsample=3)

EXPERIMENTAL

In [ ]:
v.plot_skeleton_on_img(all_pafs.numpy(),image_by_id(sample_tensor[0]))

In [ ]:
sample_tensor[3].shape[0]